In [ ]:
import os
import glob
import numpy as np
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import gradio as gr
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import pandas as pd

In [ ]:
#fetching the dataset
files = glob.glob(
    r"C:\Users\hitan\.cache\huggingface\hub\datasets--next-tat--TAT-QA\**\*.json",
    recursive=True
)

files

['C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_dev.json',
 'C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_test.json',
 'C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_test_gold.json',
 'C:\\Users\\hitan\\.cache\\huggingface\\hub\\datasets--next-tat--TAT-QA\\snapshots\\c96247f5077eac447f63527fd3dcfdc58bb56d6a\\tatqa_dataset_train.json']